In [629]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re
import time
import pandas as pd
import os
import json
import jieba

In [575]:
def get_html(url):
    #模擬訪問頁面的函數
    try:
        user_agent = 'Mozilla/5.0'
        resp = requests.get(url, headers={'User-Agent': user_agent}, timeout = 30) #回傳為一個request.Response的物件
        resp.endcoding = 'utf8'
        #response = resp.content.decode()
        #c = resp.status_code
        return resp.text
    except:
        return 'ERROR'

In [576]:
#檢查一下get_html沒有問題
#url = 'https://tw.stock.yahoo.com/news/%E5%90%84%E5%A0%B1%E8%A6%81%E8%81%9E-%E8%81%AF%E7%99%BC%E7%A7%91%E6%90%B6nokia%E8%80%81%E5%AE%B6%E4%BA%BA%E6%89%8D-015907840.html'
#k = get_html(url)
#print(k)

In [612]:
def get_url_list(url):
    #函數用來找出各日財報狗搜集的個股新聞連結，並把下一期要訪問的連結放進next_url
    html = get_html(url)
    
    #找出每個新聞的url及下一個要訪問的動態網站的url
    json_parse = json.loads(html)
    source_url = []
    
    for i in json_parse["items"]:
        source_url.append(i["source_url"])
   
    '''
    pattern = '"source_url":"\w+\S+'#s://tw.stock.yahoo.com\S+ #'http\S' 這裡不知為啥re沒辦法分出全部的
    source_url = re.findall(pattern, html)
    source_url1 = []
    for i in source_url:
        i = i.strip('"source_url":"')
        m = str()
        for j in i:
            if j != '"':
                m += j
            else:
                break
        source_url1.append(m)
    '''
     
    #找出下一個要訪問的url
    next_url = re.findall('"next_url":"\S+', html)
    next_url[0] = next_url[0].strip('"next_url":"')
    next_url1 = str()
    for i in next_url[0]:
        if i != '"':
            next_url1 += i
        else:
            break
            
    #找出每個新聞的日期
    url_date = []
    pattern1 = '"created_at":"\S{10}'
    url_date = re.findall(pattern1, html)
    url_date1 = []
    for i in url_date:
        i = i.strip('"created_at":"')
        url_date1.append(i)
    
    #做一個dict，讓日期對應到url
    date_url_dict = dict()
    for i in url_date1:
        if i in date_url_dict:
            date_url_dict[i].append(source_url[url_date1.index(i)])
        else:
            date_url_dict[i] = [source_url[url_date1.index(i)]]
            
    return [date_url_dict, next_url1]


In [614]:
#檢查一下get_url_list沒有問題
url = 'https://statementdog.com/api/v1/feeds?max_time=2019-02-26T08:28:24+08:00&type=news,tweet,announcement,insight_post&stock_id=413'
k = get_url_list(url)
print(k)

[{'2019-02-26': ['https://tw.stock.yahoo.com/news/%E7%86%B1%E9%96%80%E6%97%8F%E7%BE%A4-5g-%E6%91%BA%E7%96%8A%E6%89%8B%E6%A9%9F%E9%A1%8C%E6%9D%90%E9%80%A3%E7%99%BC-%E8%8F%AF%E7%82%BA%E9%8F%88%E5%96%8A%E8%A1%9D-234918390.html', 'https://tw.stock.yahoo.com/news/%E7%86%B1%E9%96%80%E6%97%8F%E7%BE%A4-5g-%E6%91%BA%E7%96%8A%E6%89%8B%E6%A9%9F%E9%A1%8C%E6%9D%90%E9%80%A3%E7%99%BC-%E8%8F%AF%E7%82%BA%E9%8F%88%E5%96%8A%E8%A1%9D-234918390.html'], '2019-02-25': ['https://tw.stock.yahoo.com/news/2-25%E8%AD%89%E4%BA%A4%E6%89%80%E9%9B%B6%E8%82%A1%E4%BA%A4%E6%98%93%E6%88%90%E4%BA%A4%E8%82%A1%E6%95%B8%E5%89%8D20%E5%90%8D%E6%8E%92%E8%A1%8C-082635145.html', 'https://tw.stock.yahoo.com/news/2-25%E8%AD%89%E4%BA%A4%E6%89%80%E9%9B%B6%E8%82%A1%E4%BA%A4%E6%98%93%E6%88%90%E4%BA%A4%E8%82%A1%E6%95%B8%E5%89%8D20%E5%90%8D%E6%8E%92%E8%A1%8C-082635145.html', 'https://tw.stock.yahoo.com/news/2-25%E8%AD%89%E4%BA%A4%E6%89%80%E9%9B%B6%E8%82%A1%E4%BA%A4%E6%98%93%E6%88%90%E4%BA%A4%E8%82%A1%E6%95%B8%E5%89%8D20%E5%90%8D%E6%8E%92

In [605]:
def get_content(url):
    #這個函數用來取得給定yahoo新聞連結的內容
    html = get_html(url)
    soup = BeautifulSoup(html, 'html.parser')
    contents = soup.find_all('p')
    contents1 = str()
    for i in contents:
        contents1 += i.text
    '''
    不知道為啥這裡Xpath不能找出正確的東西
    tree = etree.HTML(html)
    result = etree.tostring(tree, pretty_print = True, method = "html")
    text = str()
    for i in tree.xpath('//*[@id="aritcletable"]/tbody/tr[1]/td/table/tbody/tr'):
        text = i.xpath('string(.)')
    ''' 
    return contents1

In [606]:
#檢查get_content沒有問題
a = "https://tw.stock.yahoo.com/news/%E7%A7%91%E6%8A%80-%E5%AE%89%E8%AC%80-%E8%81%AF%E7%99%BC%E7%A7%915g%E5%B0%87%E5%A4%A7%E6%94%BE%E7%95%B0%E5%BD%A9-004647873.html"
text = get_content(a)
print(text)



【時報-台北電】聯發科 （2454） 在手機市場能站上國際舞台，成為全球前段班腳色，除了聯發科集團1.6萬名員工的共同努力之外，晶片當中的基礎架構更是由矽智財大廠安謀（ARM）所提供。ARM矽智財產品事業群主管Rene Haas表示，聯發科與ARM已經合作超過20年，看好聯發科未來將能持續在5G市場上大放異彩。　聯發科每年出貨晶片數量可觀，一年至少有15億部終端裝置導入聯發科的晶片在其中，更造就聯發科在全球IC設計廠當中名列前十的地位，與國際大廠高通（Qualcomm）、輝達（NVIDIA）、超微（AMD）齊名。　然而，這其中的關鍵除了在於聯發科工程研發技術強大，同時CPU矽智財大廠ARM也在背後默默推了一手。簡單來說，由於晶片開發過程就好比將矽智財透過積木拼接方式把產品開發出來，而ARM正是提供聯發科矽智財的廠商。　Rene Haas指出，ARM與聯發科的合作歷史長達20年之久，早在聯發科仍把機上盒及光碟機視為營收主力的時代就已經展開合作，隨著時代變遷，聯發科已經將進入5G時代，看好聯發科未來在5G佈局完整之下，有機會持續成長。　Rene Haas認為，聯發科在技術研發上具有領先地位，不論是5G、物聯網（IoT）等市場皆是如此，因此在未來的機器學習（Machine Learning）及深度學習（Deep Learning）等應用，看好聯發科能夠快速掌握先機。　由於ARM目前在行動裝置市佔率至少高達九成，穩坐龍頭老大地位，甚至在消費性電子及家電市場也擁有絕對的領先地位，更是全球微控制器（MCU）及行動晶片的矽智財供應商，因此ARM可以說是IC設計產業的前段供應商。　對於台灣IC設計產業，Rene Haas指出，台灣IC設計產業目前為全球前三大市場之一，且擁有完整的晶圓代工、封測等半導體供應鏈，在零組件組裝更有全球領先的ODM／OEM廠，但人口僅兩千三百萬人，這是相當令人驚嘆的成就。(新聞來源：工商時報─蘇嘉維／英國劍橋26日專電)台股資料來源臺灣證券交易所、臺灣期貨交易所及財團法人中華民國證券櫃檯買賣中心。國際股市資料來源請參考Yahoo Finance。使用Yahoo奇摩股市服務前，請您詳閱相關使用規範與聲明。


In [625]:
#把爬到的新聞同日的組合在一起，並存到一個檔案(暫時先用字典，週數也暫時先用兩週)
first_url = 'https://statementdog.com/api/v1/feeds?max_time=2019-02-26T08:28:24+08:00&type=news,tweet,announcement,insight_post&stock_id=413'
content_dict = dict()
url = first_url
number_of_week = 2
#底下處理第一週
for key in get_url_list(url)[0]:
        try:
            for i in get_url_list(url)[0][key]:
                single_text = get_content(i)
                if key in content_dict:
                    content_dict[key] += single_text
                else:
                    content_dict[key] = single_text
        except:
            content_dict[ERROR] = 1 
#第一週以後
for i in range(number_of_week - 1):
    url = get_url_list(url)[1]
    for key in get_url_list(url)[0]:
        try:
            for i in get_url_list(url)[0][key]:
                single_text = get_content(i)
                if key in content_dict:
                    content_dict[key] += single_text
                else:
                    content_dict[key] = single_text
        except:
            content_dict[ERROR] = 1 

print(content_dict)

{'2019-02-26': '\n【時報-台北電】中美貿易戰消息越來越樂觀，市場再度重燃對大陸電信設備大廠華為的信心，挾帶5G題材及摺疊手機發表，25日台灣華為概念股多頭噴發，光聖 (6442) 、聯發科 (2454) 、統新 (6426) 等漲逾6％，股價報喜。\u3000美國總統川普宣布延後3月1日對大陸商品提高關稅的計畫，全球股市多頭激昂，大盤指數25日漲68點、0.66％收10,390點，連4日收在最高點。\u3000原先被中美貿易戰打趴的華為也重振威風，市場再度對即將爆發的5G建置大力埋單，加上近日於MWC展發布5G摺疊手機Mate X，題材連發，華為概念股雨露均霑。\u3000產業法人指出，中興通訊、華為事件一度導致市場悲觀，然5G發展趨勢並未因此停滯，全球5G基站建置仍以華為占比三成以上馬首是瞻，加上中美貿易戰氛圍再度轉為樂觀，有利台灣光通訊族群聯亞、華星光等評價提升。\u3000另外，圍繞MWC展的焦點莫過於5G折疊手機，華為又順勢推出搭載7奈米5G晶片的Mate X，雖然價格昂貴，仍搶盡風采，與手機相關的華為概念股具有想像空間。\u3000觀察25日股價漲幅強勁的華為概念股，光通訊的光聖、統新、華星光等榜上有名，與手機相關的聯發科、日月光投控等也有3％以上的漲幅，其中，外資除調節統新、旺宏外，其餘買超，尤其買超聯發科、欣興6千餘張最多。\u3000光聖2018年本業逐漸好轉，法人預估每股盈餘可望達到2.2元，加上2019年1月營收3.12億元，年增34.3％、月增56.7％，26日放量大漲7.43％收43.4元，成交量暴增逾19倍。（新聞來源：工商時報─記者涂憶君／台北報導）台股資料來源臺灣證券交易所、臺灣期貨交易所及財團法人中華民國證券櫃檯買賣中心。國際股市資料來源請參考Yahoo Finance。使用Yahoo奇摩股市服務前，請您詳閱相關使用規範與聲明。\n【時報-台北電】中美貿易戰消息越來越樂觀，市場再度重燃對大陸電信設備大廠華為的信心，挾帶5G題材及摺疊手機發表，25日台灣華為概念股多頭噴發，光聖 (6442) 、聯發科 (2454) 、統新 (6426) 等漲逾6％，股價報喜。\u3000美國總統川普宣布延後3月1日對大陸商品提高關稅的計畫，全球股市多頭激昂，大盤指數25日漲68點、0.66％收10,390點，連4日收在最高點。\u30

In [643]:
#jieba.set_dictionary('dict.txt.big')
def sort_dict_by_values(d):
    return(sorted(d.items(), key=lambda x: x[1], reverse=True))
date_parsed_word = dict()
for key in content_dict:
    jieba_parse_eg = jieba.cut(content_dict[key], cut_all=False)
    #print(" / ".join(jieba_parse_eg))
    word_count = dict()
    for i in jieba_parse_eg:
        if i in word_count:
            word_count[i] += 1
        else:
            word_count[i] = 1
    
    date_parsed_word[key] = sort_dict_by_values(word_count)
    
print(date_parsed_word)

{'2019-02-26': [('，', 68), ('、', 24), ('的', 22), ('。', 20), (' ', 18), ('華為', 14), ('5G', 14), ('％', 12), ('\u3000', 12), ('手機', 10), ('日', 8), ('概念股', 8), ('等', 8), ('市場', 6), ('再度', 6), ('對', 6), ('25', 6), ('光聖', 6), ('(', 6), (')', 6), ('統新', 6), ('月', 6), ('點', 6), ('加上', 6), ('法人', 6), ('與', 6), ('相關', 6), ('來源', 6), ('時報', 4), ('台北', 4), ('大陸', 4), ('題材', 4), ('及', 4), ('摺', 4), ('疊', 4), ('多頭', 4), ('聯發科', 4), ('逾', 4), ('6', 4), ('3', 4), ('1', 4), ('收', 4), ('也', 4), ('建置', 4), ('於', 4), ('MWC', 4), ('Mate', 4), ('X', 4), ('仍', 4), ('以上', 4), ('光通', 4), ('訊', 4), ('華', 4), ('星光', 4), ('買', 4), ('年', 4), ('元', 4), ('資料', 4), ('臺', 4), ('交易所', 4), ('股市', 4), ('請', 4), ('Yahoo', 4), ('使用', 4), ('\n', 2), ('【', 2), ('-', 2), ('電', 2), ('】', 2), ('中美貿易戰', 2), ('消息', 2), ('越來', 2), ('越樂觀', 2), ('重燃', 2), ('電信', 2), ('設備', 2), ('大廠', 2), ('信心', 2), ('挾帶', 2), ('發表', 2), ('台灣華為', 2), ('噴發', 2), ('6442', 2), ('2454', 2), ('6426', 2), ('漲', 2), ('股價', 2), ('報喜', 2), ('美國', 2), ('總', 2)

In [ ]:
#之後可以自訂辭典調整詞，並用每日報酬率與當日新聞出現的高頻詞彙做統計，看哪些是與股價日報酬率有關連